## Import liberay and set connection with database with database cradensial.

In [21]:
import os
from sqlalchemy import create_engine, text, inspect
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

# Database connection details
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')
gill_api = os.getenv('GILL_API_KEY')

DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
engine = create_engine(DATABASE_URL)

### Database Connecting test query

In [4]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT DATABASE()"))
        db_name = result.fetchone()[0]
        print(f"Connected to database: {db_name}")
except Exception as e:
    print(f"Error: {e}")

Connected to database: itt_master_contents


### Print All table list in the database for check connection.

In [8]:
try:
    inspector = inspect(engine)
    tables = isnpector.get_table_names()
    for table in tables:
        print(table)
except Execption as e:
    print(f"Error: {e}")

hotel_info_all
hotels_info_with_gidestination_code
vervotech_ProviderFamily
vervotech_hotel_list
vervotech_hotel_map_new
vervotech_hotel_map_update
vervotech_mapping
vervotech_update_data_info


### Function for get table specifiq column. Here get PostalCode, CountryCode, CuntryName

In [19]:
def get_data_from_table(table, country_code, engine):
    query = f"SELECT CountryCode, GiDestinationId FROM {table} where GiDestinationId != 'NULL' AND CountryCode = '{country_code}'"
    df = pd.read_sql(query, engine)
    return df

data = get_data_from_table(table = 'hotels_info_with_gidestination_code', country_code = 'AE', engine=engine)

print(data)
    
    

   CountryCode GiDestinationId
0           AE            3029
1           AE            3029
2           AE            3046
3           AE            3027
4           AE          182814
5           AE           28431
6           AE            3040
7           AE            3037
8           AE            3039
9           AE           99836
10          AE           71919
11          AE            3042
12          AE            3032
13          AE            3045
14          AE            3048
15          AE            3049
16          AE          142303


### Input data a table where CountryCode = "AE"

In [20]:
def update_country_code(df, target_table, engine):
    """ Updates the CountryCode in the target table based on GiDestinationId."""

    # Iterate over each row in the DataFrame
    with engine.begin() as connection:
        for _, row in df.iterrows():
            country_code = row['CountryCode']
            gi_destination_id = row['GiDestinationId']
            
            query = text(f"""
                UPDATE {target_table}
                SET CountryCode = :country_code
                WHERE GiDestinationId = :gi_destination_id
            """)
            
            connection.execute(query, {
                "country_code": country_code,
                "gi_destination_id": gi_destination_id
            })

data = get_data_from_table(
    table_name='hotels_info_with_gidestination_code', 
    country_code='AE', 
    engine=engine
)

# Update the 'hotel_info_all' table based on data from the DataFrame
update_country_code(data, target_table='hotel_info_all', engine=engine)

print("CountryCode updated successfully.")


CountryCode updated successfully.


### Get column_info with where CountryCode = "AE"

In [64]:
def only_select_column_info(table, column, country_code, engine):
    """Fetch distinct values from a specified column in a given table."""
    try:
        query = f"SELECT {column} FROM {table} WHERE StatusUpdateHotelInfo != 'Done Json' OR StatusUpdateHotelInfo IS NULL AND CountryCode = '{country_code}';"
        df = pd.read_sql(query, engine)
        

        unique_values = df[column].unique()
        # print(len(uniqe_value))
        
        # len_list = len(list)
        # print(len_list)

        all_values = df[column].tolist()
        
        # Count unique values
        unique_values = df[column].unique()
        # print(f"Number of unique values: {len(unique_values)}")
        
        # # Find duplicates
        # duplicates = df[df[column].duplicated()][column].tolist()  
        # duplicate_counts = df[column].value_counts()[df[column].value_counts() > 1]  

        # print(f"Total values: {len(all_values)}")
        # print(f"Duplicate values: {duplicates}")
        # print(f"Duplicate counts:\n{duplicate_counts}")

        return unique_values
    except Exception as e:
        print(f"Error fetching column info: {e}")
        return [], pd.Series()
        
data = only_select_column_info('hotel_info_all', 'SystemId', 'AE', engine)
print(data)

[1356686 1353543 1283949 ...  114932  118590   20200]


### Check all unique key with count id.

In [ ]:
data = only_column_info('hotel_info_all', 'SystemId', 'AE', engine)

for _, i in enumerate(data):
    print(f"index: {_} systemId: {i} ")

In [75]:
import time
import requests
import json
from datetime import datetime

In [76]:
def fetch_hotel_info_by_systemId(systemId):
    """Fetch hotel information by system ID using synchronous requests."""
    url = "https://api.giinfotech.ae/api/Hotel/HotelInfo"
    payload = json.dumps({"hotelCode": str(systemId)})
    headers = {
        'ApiKey': gill_api,
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(url, headers=headers, data=payload)
        if response.status_code == 200:
            response_data = response.json()
            if response_data["isSuccess"]:
                hotel_info = response_data["hotelInformation"]
                return hotel_info
            else:
                print(f"No hotel information found for systemID: {systemId}")
        else:
            print(f"Failed to fetch data for systemID {systemId}: {response.status_code}")
    except Exception as e:
        print(f"Error fetching data for system ID {systemId}: {e}")
    
    return []

In [77]:
def update_hotel_info(systemId, hotel_info_json_data, status_update, engine):
    """Update hotel information in the database."""

    if isinstance(hotel_info_json_data, str):
        hotel_info_json_data = json.loads(hotel_info_json_data)
    
    country_code = hotel_info_json_data.get("address", {}).get("countryCode")
    zip_code = hotel_info_json_data.get("address", {}).get("zipCode")
    country_name = hotel_info_json_data.get("address", {}).get("countryName")
    
    query = text("""
        UPDATE hotel_info_all
        SET HotelInfo = :HotelInfo,
            StatusUpdateHotelInfo = :StatusUpdateHotelInfo,
            CountryCode = :CountryCode,
            ZipCode = :ZipCode,
            CountryName = :CountryName
        WHERE SystemId = :SystemId
    """)
    
    with engine.begin() as connection:
        connection.execute(query, {
            "HotelInfo": json.dumps(hotel_info_json_data),
            "StatusUpdateHotelInfo": status_update,
            "CountryCode": country_code,
            "ZipCode": zip_code,
            "CountryName": country_name,
            "SystemId": systemId
        })
        print(f"Updated SystemId: {systemId} with Status: {status_update}.")

In [ ]:

def main():
    start_time = time.time()
    formatted_start_time = datetime.fromtimestamp(start_time).strftime("%I:%M %p")  
    print(f"Start Time: {formatted_start_time}")

    table = 'hotel_info_all'
    system_ids = only_select_column_info('hotel_info_all', 'SystemId', 'AE', engine)

    for index, systemId in enumerate(system_ids, start=1):
        hotel_info = fetch_hotel_info_by_systemId(systemId)
        if hotel_info:
            status_update = "Done Json"
            update_hotel_info(systemId, json.dumps(hotel_info), status_update, engine)
            print(f"Update system Id Done: No:{index} ----- {systemId}")
        else:
            status_update = "Not found json"
            update_hotel_info(systemId, json.dumps({}), status_update, engine)
            print(f"Update system Not found json: No:{index} ---------------------------------------------------------------- {systemId}")

    end_time = time.time()  
    formatted_end_time = datetime.fromtimestamp(end_time).strftime("%I:%M %p")
    print(f"END time: {formatted_end_time}")

    total_time = end_time - start_time

    # print(f"Total time taken for updates: {total_time:.2f} seconds")
    # Convert total time to hours, minutes, and seconds

    hours = int(total_time // 3600)
    minutes = int((total_time % 3600) // 60)
    seconds = int(total_time % 60)

    print(f"Total time taken for updates: {hours} hours, {minutes} minutes, {seconds} seconds")



In [79]:
if __name__ == "__main__":
    main()


KeyboardInterrupt: 